# Student habits vs Academic performance

## Enunciado

Utilizando como base o dataset [Student habits v Academic performance](../datasets/student_habits_performance.csv):

1. Apenas com a observação dos dados por meio de tabelas e dataframe, apresente suas hipóteses;

2. Realize uma EDA da forma que julgar mais adequada;

3. Com base na EDA realizada, revise as suas hipóteses a respeito dos dados;

4. Construa um algoritmo de K-Means from scratch utilizando a linguagem Python

5. Apresente a justificativa de forma discursiva e por meio de cálculos, o valor de K utilizado;

6. Revise as hipóteses levantadas nos exercícios 1 e 2 e com base no algoritmo desenvolvido, apresente análise crítica a respeito das informações extraídas dos dados caso o algoritmo refute ou confirme as suas hipóteses;

Tarefas Extra:

7. Exercite sua habilidade de pesquisa e curadoria de informações e a partir de fontes próprias, escreva, com suas palavras, de maneira teórica e com o aprofundamento que achar adequado os algoritmos DBSCAM e Hierarchical Clustering;

8. Exercite sua habilidade de pesquisa e curadoria de informações e a partir de fontes próprias, escreva, com suas palavras, de maneira teórica e com o aprofundamento que achar adequado qual o algoritmo SotA para clustering. Apresente as referências utilizadas;

https://chatgpt.com/share/6821eda7-00f4-8006-9abd-43bdd7df4be1


## Setup


### Importando pacotes e dados


### Declarações de funções


## Hipóteses inicias a partir de inspeção de dados


## EDA


### Revisão das Hipóteses


## Implementação de K-Means from scratch

https://www.youtube.com/watch?v=lX-3nGHDhQg

https://www.youtube.com/watch?v=5w5iUbTlpMQ

https://medium.com/@avijit.bhattacharjee1996/implementing-k-means-clustering-from-scratch-in-python-a277c23563ac

https://github.com/aihubprojects/Machine-Learning-From-Scratch/blob/master/K-Means%20from%20Scratch.ipynb?utm_source=chatgpt.com


## Justificativa do valor de $K$

Métodos para determinar K:

- Elbow Method (WCSS vs.K): procure o “cotovelo” na curva de soma de quadrados dentro do cluster [link](https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/?utm_source=chatgpt.com)

- Silhouette Score: avalia coesão e separação dos clusters; valores mais altos indicam clusters bem definidos [link](https://www.analyticsvidhya.com/blog/2021/05/k-mean-getting-the-optimal-number-of-clusters/?utm_source=chatgpt.com)

- Gap Statistic: compara inércia do seu dataset com dataset de referência aleatório.


## Análise crítica final


## Tarefas Extra


### DBSCAN

https://www.geeksforgeeks.org/dbscan-clustering-in-ml-density-based-clustering/?utm_source=chatgpt.com

https://en.wikipedia.org/wiki/DBSCAN?utm_source=chatgpt.com


### Hierarchical Clustering

https://www.ibm.com/think/topics/hierarchical-clustering?utm_source=chatgpt.com

https://en.wikipedia.org/wiki/Hierarchical_clustering?utm_source=chatgpt.com


### SotA para Clustering

https://peerj.com/articles/cs-2286.pdf?utm_source=chatgpt.com

https://www.researchgate.net/publication/358814868_A_comprehensive_survey_of_clustering_algorithms_State-of-the-art_machine_learning_applications_taxonomy_challenges_and_future_research_prospects?utm_source=chatgpt.com

https://medium.com/%40sina.nazeri/comparing-the-state-of-the-art-clustering-algorithms-1e65a08157a1?utm_source=chatgpt.com
